Welcome on this second notebook, i will show you how i accomplished on extracting the features from a real photo. Also i will apply the color correction on the photo, showing you the possibilities and the limitations of this process.

In [3]:
import cv2
import numpy as np
import sys
from pyzbar.pyzbar import decode
import colour
import pandas as pd
import sklearn
#Reference colors taken from Macbeth Colour Checker
#We need them for the color correction
real_colors = [
[52.0, 52.0, 52.0],
[85.0, 85.0, 85.0],
[121.0, 122.0, 122.0],
[160.0, 160.0, 160.0],
[200.0, 200.0, 200.0],
[242.0, 243.0, 243.0],
[161.0, 133.0, 8.0],
[149.0, 86.0, 187.0],
[31.0, 199.0, 231.0],
[60.0, 54.0, 175.0],
[73.0, 148.0, 70.0],
[150.0, 61.0, 56.0],
[46.0, 163.0, 224.0],
[64.0, 188.0, 157.0],
[108.0, 60.0, 94.0],
[99.0, 90.0, 193.0],
[166.0, 91.0, 80.0],
[44.0, 126.0, 214.0],
[179.0, 189.0, 103.0],
[177.0, 128.0, 133.0],
[67.0, 108.0, 87.0],
[157.0, 122.0, 98.0],
[130.0, 150.0, 194.0],
[68.0, 82.0, 115.0]
]

In [4]:
#Change the absolute path to read the prototype
path = "C:/Users/User Pc/Desktop/Progetto OpenCV/All notebooks/real_photo.jpeg"
img = cv2.imread(path)


"""
Method to incapsulate the imshow, just for prettify the code
"""
def imshow(img, name = "img"):
    cv2.imshow(name,img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

"""
This method is the preprocessing pipeline. I needed to 
change it from the classic method because the 
first implementation wasn't enough efficient on 
extracting the corners without introducing noise inside
its output.
"""
def bilateralFilteringPreprocessing(img):
    
    #Get only one channel
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #Blur the image using gaussian blur
    imgBlur = cv2.GaussianBlur(gray,(7,7),2)
    
    # Smoothing without removing edges.
    gray_filtered = cv2.bilateralFilter(gray, 9, 60, 120)
  
    # Applying the canny filter
    edges_filtered = cv2.Canny(gray_filtered, 30, 50) 

    return edges_filtered

imshow(img)
imshow(bilateralFilteringPreprocessing(img))


Extract the QrCode coords and remove it from the photo, defining two different images:

1) References, for color correction

2) Chemical pads, for the result of the analysis

I've tried different photos, but i will show only this one for now, because sometimes the shadows inside the photo can be problematic for the qrCode decoding. I think i can try to change the pipeline not using the qrCode, but i need to redesign the operations we need.


In [5]:
"""
Given the image in input, extract the QrCode and return 
the upper part of the image and the lower part
"""
def qrCodeExtraction(img):
    
    #Method of pyzbar that decodes the image
    qrCode = decode(img)[0]
    
    #Contains different basic object, we need the polygon
    #because it contains the four points of the bounding box
    polygon = qrCode.polygon
    
    #Find min and max of the y_coords of the qrCode
    qrCode_min_y = min(value.y for value in polygon)
    qrCode_max_y = max(value.y for value in polygon)
    
    #Divide the img in two different ones
    references = img[:qrCode_min_y-10].copy()
    chemical_pads = img[qrCode_max_y:].copy()
    
    return qrCode_min_y, qrCode_max_y,references, chemical_pads

qrCode_min_y, qrCode_max_y, references, chemical_pads = qrCodeExtraction(img)

Method useful to study the different contours that OpenCV finds inside the image. Is important to notice that bigger images bring with them bigger noises, so is necessary the definition of some practices useful to remove the noises. 

In [ ]:
"""
Method that return all the lines in the tester and also the min and max points 
from them, so that we can crop the image
"""
def getContourWithDefinedNumber(img, numberDesired):
    
  imgContour = img.copy()
  imgCanny = bilateralFilteringPreprocessing(img)
  #Store couple [[x_Start,y_Start], [X_End,y_End]] for each contour that we found
  #with the desired number of boundary points
  output = []

  #Extract all the contours from the img
  contours,hierarchy = cv2.findContours(imgCanny,cv2.RETR_LIST,
                                        cv2.CHAIN_APPROX_NONE)
  #For each contour found
  for cnt in contours:
        
    #Calculate the approximative Polygon to extract only the lines from the photo
    peri = cv2.arcLength(cnt,True)
     
    #Find the approximative Polygon based on the perimeter and the contour
    #approx is a list of points that define the polygon
    approx = cv2.approxPolyDP(cnt,0.02 * peri,True)
    
    #Number of corners of the polygon
    objCor = len(approx)
    
    #Return only the contours that have only the desired value
    if (objCor == numberDesired):
      
      #Draw the desired contour
      cv2.drawContours(imgContour,cnt,-1,(255,0,0),1)
      
      #Define the boundingRect so that, in the case of a line o a square, we have
      #The left coordinate, the width and the height
      x,y,w,h = cv2.boundingRect(approx)
      
      #Just for us to visualize the output
      cv2.rectangle(imgContour,(x,y),(x+w,y+h),(0,255,0),1)
        
      output.append(approx)
  #Show the output
  imshow(imgContour,str(numberDesired))

#Extract the lines from the tester using Canny Algorithm and cv2.drawContours
getContourWithDefinedNumber(references,4)
getContourWithDefinedNumber(chemical_pads,4)



In [ ]:
"""
Object that packs different methods useful for the color extraction from 
each bounding box
"""
class boxObject():
    
    #Init method
    def __init__(self,x,y,w,h):
        self.x = x
        self.y = y
        self.h = h
        self.w = w
        self.area = w*h
    
    #Given an image and a radius(predefine at 0.3), the method return the RGB of the area defined by the radius.
    def colorInside(self,img, radius = 0.1):
        
        #Extract the bounding Rect from the photo
        imageInside = img[self.y:self.y+self.h,self.x:self.x+self.w].copy()
        
        #Define the radius based on the w and h of the box
        radiusX = round(radius * self.w) 
        radiusY = round(radius * self.h)
        
        #Find the center from radiusX and radiusY
        center = imageInside[radiusX : 2*radiusX, radiusY : 2*radiusY]
        
        #Return the BGR of the area
        return [round(np.mean(center[:,:,0])),round(np.mean(center[:,:,1])),round(np.mean(center[:,:,2]))]
    
    def print(self):
        print(self.x,self.y,self.h,self.w)
    
    #Check if other is a duplicate of self
    def is_duplicate_of(self, other):
        x_dist = abs(self.x - other.x)
        y_dist = abs(self.y - other.y)
        area_div = self.area/other.area
        return (x_dist < 10 and 
                y_dist < 10 and 
                area_div > 0.9)

In [ ]:
"""
Given an image and the number of vertex desired, returns a list
that contains the (x,y,w,h) of each elements that has pol edges
Used to extract the coordinates of each square
"""
def getPolygonCoords(img,pol = 2):
  imgContour = img.copy()
  #Preprocessing the image
  imgCanny = bilateralFilteringPreprocessing(img)
    
  #Find the contours so that we can choose the ones desired
  contours,hierarchy = cv2.findContours(imgCanny,cv2.RETR_LIST,
                                        cv2.CHAIN_APPROX_NONE)
  #Output array
  boxes = []
    
  #For each contours
  for cnt in contours:
    
    #Find the perimeter of the contour
    peri = cv2.arcLength(cnt,True)
    
    #Find the approximative Polygon based on the perimeter and the contour
    #approx is a list of points that define the polygon
    approx = cv2.approxPolyDP(cnt,0.02 * peri,True)
    
    #If has the desired number of corners
    if (len(approx) == pol):
      
      #Draw for each contour it's shape
      cv2.drawContours(imgContour,cnt,-1,(0,255,0),1)
    
      #Find the bounding rect of the approximative polygon
      x,y,w,h = cv2.boundingRect(approx)
        
      #Append it to the output array
      boxes.append(boxObject(x,y,w,h))
  
  #Just to show the different boxes extracted
  imshow(imgContour)
  return boxes

We can use the different lines inside the tester to cut it in more smaller parts, useful to extract the references and the chemical pads.
As you can see running this cell, there is a problem with the background: it is too bright, so is difficult to find the colors inside the references that don't have enough contrast with the background.
Two solutions:

1) Change background color, but i think is terrible to have a black/dark gray tester.

2) Give a contour to each reference color, need to try it, but i think it willgenerate more duplicate for each box, but  created a function that removes the duplicates, so i just need to test it.

In [7]:
"""
Given an image in input, returns the (min,max) of the y_coords of the lines inside the photo.
Useful to isolate the references\chemical_pads
"""
def getLinesCoords(img):
    l = getPolygonCoords(img,2)
    #Choose only the horizontal lines inside the img
    lines = [value for value in l if(value.h/value.w <0.1)]
    #Compact way to iterate,using the generators functions
    y_max = max(value.y for value in lines)
    y_min = min(value.y for value in lines)
    return y_min,y_max

#Find the two lines that define the reference space
#Inside the tester
y_min_references,y_max_references = getLinesCoords(references)
references_isolated = references[y_min_references:y_max_references,:] 

#Extract the squares and remove the noise
references_coords = getPolygonCoords(references_isolated,4)
references_squares = [value for value in references_coords if(value.area >10)]



Same problem on the references, need to change the color on the real starting hexadecimal of each chemical pad to see if the proble persists.

In [8]:
#The exact same thing that i've done for the references, but 
#for the chemical pads
y_min_chemical_pads,y_max_chemical_pads = getLinesCoords(chemical_pads)
chemical_pads_isolated = chemical_pads[y_min_chemical_pads:,:] 

chemical_coords = getPolygonCoords(chemical_pads_isolated,4)
chemical_squares = [value for value in chemical_coords if(value.area >10)]


We need to remove the duplicated contours that openCv finds inside the image

In [9]:
"""
Given a list of boxObjects in input, return a list without all the duplicates
"""
def remove_box_duplicates(squares):
    set_sq = list()
    set_sq.append(squares[0])
    #Iterate on each different square and verify that is not a duplicate
    for i in range(1,len(squares)):
        box = squares[i]
        duplicated = False
        #Iterate on the set to verify the uniqueness of box
        for sq in set_sq:
            duplicated = duplicated or box.is_duplicate_of(sq)
            if duplicated:
                break;
        if not duplicated:
             set_sq.append(box)
    return set_sq

set_references_sq = remove_box_duplicates(references_squares)
set_chemical_sq = remove_box_duplicates(chemical_squares)

Visualization of the different boxes extracted, with a number that indicates their order, from downer left to upper right

In [16]:
def showIndexedSquares(img, set_sq):
    imgContour = img.copy()
    i = 0
    for value in set_sq:
        x = value.x
        y = value.y
        h = value.h
        w = value.w
        #Just for us to visualize the output
        cv2.rectangle(imgContour,(x,y),(x+w,y+h),(0,255,0),1)
        cv2.putText(
         imgContour, #numpy array on which text is written
         str(i), #text
         (x,y), #position at which writing has to start
         cv2.FONT_HERSHEY_SIMPLEX, #font family
         1, #font size
         (0,255,0), #font color
         3) #font stroke
        i = i+1
    imshow(imgContour)


In [17]:
showIndexedSquares(references_isolated, set_references_sq)
showIndexedSquares(chemical_pads_isolated, set_chemical_sq)

481 22
511 58


Number taken from the photo + extracted by hand from the photo(just to try, because we need to change the background color)

In [12]:
colors_extracted = list()
for value in set_references_sq:
    colors_extracted.append(value.colorInside(references_isolated))
colors_extracted.insert(5,[169,169,167])
colors_extracted.insert(8,[188,158,32])

Color correction, as you can see the image changes drastically, but the reference colors don't change too much, need to test more this process or to check if exists better way to correct the colors of the photo. You can also notice that the shadow in the photo introduce a very big noise, so i think could be useful study a way to use the smartphone flashlight in a useful way.

In [13]:
cali_img = morph = img.copy()

for im in cali_img:
    im[:] = colour.colour_correction(im[:], colors_extracted, real_colors,'Finlayson 2015')

cv2.waitKey(0)
imshow(cali_img)

Trying to use PLS Regression, similar results to the Finlayson Algorithm

In [ ]:
from sklearn.cross_decomposition import PLSRegression

pls = PLSRegression(n_components = 3, scale = True, copy = True, tol = 1e-6)
                    
pls.fit(colors_extracted,real_colors)
pls.score(colors_extracted,real_colors)

In [ ]:
cali_img = morph = img.copy()

for im in cali_img:
    im[:] = pls.predict(im[:])

imshow(cali_img)

Extract a second time the colors and the boxes from the photo, using the same points we found earlier on the original img.

In [ ]:
new_references = cali_img[:qrCode_min_y-10].copy()
new_chemical_pads = cali_img[qrCode_max_y:].copy()

new_references_isolated = new_references[y_min_references:y_max_references,:] 
new_chemical_pads_isolated = new_chemical_pads[y_min_chemical_pads:]

In [ ]:
imshow(new_references_isolated)
imshow(new_chemical_pads_isolated)

To see how the colors changed, i printed the old and new colors from the two images, the real one and the corrected one

In [19]:
for value in set_chemical_sq:
    
    print("Old")
    print(value.colorInside(chemical_pads_isolated))
    
    print("New")
    print(value.colorInside(new_chemical_pads_isolated))

    print("**********")

Old
[17, 88, 111]
New
[42, 120, 121]
**********
Old
[88, 76, 82]
New
[143, 126, 136]
**********
Old
[20, 38, 123]
New
[112, 70, 141]
**********
Old
[108, 105, 103]
New
[149, 157, 168]
**********
Old
[28, 19, 115]
New
[142, 52, 139]
**********
Old
[99, 102, 54]
New
[114, 157, 110]
**********
Old
[60, 77, 105]
New
[114, 119, 142]
**********
Old
[34, 34, 43]
New
[93, 76, 64]
**********
Old
[72, 61, 110]
New
[154, 105, 158]
**********


Some datas about the color correction

In [20]:
new_colors_extracted = list()
for value in set_references_sq:
    new_colors_extracted.append(value.colorInside(new_references_isolated))
new_colors_extracted.insert(5,["not found because openCV can't identify a border"])
new_colors_extracted.insert(8,["not found because openCV can't identify a border"])

for i in range(len(real_colors)):
        print("Color with index " + str(i))
        print("Old")
        print(colors_extracted[i])

        print("New")
        print(new_colors_extracted[i])
        
        print("Desired")
        print(real_colors[i])
        
        try:
            dist = np.array(new_colors_extracted[i]) - np.array(real_colors[i])
        except:
            dist = "There is no number"
        print("Distance from the desired ones")
        print(dist)

        print("**********")

Color with index 0
Old
[36, 55, 64]
New
[83, 95, 86]
Desired
[52.0, 52.0, 52.0]
Distance from the desired ones
[ 31.  43.  34.]
**********
Color with index 1
Old
[32, 44, 53]
New
[84, 84, 72]
Desired
[85.0, 85.0, 85.0]
Distance from the desired ones
[ -1.  -1. -13.]
**********
Color with index 2
Old
[45, 46, 54]
New
[102, 89, 82]
Desired
[121.0, 122.0, 122.0]
Distance from the desired ones
[-19. -33. -40.]
**********
Color with index 3
Old
[68, 68, 69]
New
[116, 115, 109]
Desired
[160.0, 160.0, 160.0]
Distance from the desired ones
[-44. -45. -51.]
**********
Color with index 4
Old
[131, 143, 145]
New
[161, 197, 224]
Desired
[200.0, 200.0, 200.0]
Distance from the desired ones
[-39.  -3.  24.]
**********
Color with index 5
Old
[169, 169, 167]
New
["not found because openCV can't identify a border"]
Desired
[242.0, 243.0, 243.0]
Distance from the desired ones
There is no number
**********
Color with index 6
Old
[80, 66, 34]
New
[117, 119, 80]
Desired
[161.0, 133.0, 8.0]
Distance from th

The results of the color correction aren't very exciting,but we have at least shortened the distance from the desired colors and the extracted ones. I think we need to change approach on the color correction.